In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Day 19

In [2]:
class Scanner:
    def __init__(self, idx, locations):
        self.index = idx
        self.beacons = [ tuple([int(i) for i in l.split(',')]) for l in locations]
        self.scanner_location = [0,0,0]
    
    def get_beacons(self):
        return self.beacons
    
    def set_scanner_location(self, location):
        self.scanner_location = location
    
    def rotate_beacons(self, rot_type=0):
        self.beacons = self.perform_rotation(rot_type)
        
    def shift_beacons(self, shift):
        beacons = self.beacons
        self.beacons = [ tuple([i[0] - i[1] for i in zip(loc,shift)]) for loc in beacons]
        
    def perform_rotation(self, rot_type=0):
        rotations = {
                    # rotate around x with X Face
                    0 : lambda x,y,z : [x, y, z] , 
                    1 : lambda x,y,z : [x,-z, y] , 
                    2 : lambda x,y,z : [x,-y,-z] ,
                    3 : lambda x,y,z : [x, z,-y] ,
                    # Negative X
                    4 : lambda x,y,z : [-x, y, -z] , 
                    5 : lambda x,y,z : [-x, z, y] , 
                    6 : lambda x,y,z : [-x,-y, z] ,
                    7 : lambda x,y,z : [-x,-z,-y] ,
    
                    # rotate around y with Y Face
                    8 : lambda x,y,z : [y, z, x] , 
                    9 : lambda x,y,z : [y,-x, z] , 
                   10 : lambda x,y,z : [y,-z,-x] ,
                   11 : lambda x,y,z : [y, x,-z] ,
                    # Negative Y
                   12 : lambda x,y,z : [-y, z, -x] , 
                   13 : lambda x,y,z : [-y, x, z] , 
                   14 : lambda x,y,z : [-y,-z, x] ,
                   15 : lambda x,y,z : [-y,-x,-z] ,
    
                    # rotate around y with Z Face
                   16 : lambda x,y,z : [z, x, y] , 
                   17 : lambda x,y,z : [z,-y, x] , 
                   18 : lambda x,y,z : [z,-x,-y] ,
                   19 : lambda x,y,z : [z, y,-x] ,
                    # Negative Z
                   20 : lambda x,y,z : [-z, x,-y] , 
                   21 : lambda x,y,z : [-z, y, x] , 
                   22 : lambda x,y,z : [-z,-x, y] ,
                   23 : lambda x,y,z : [-z,-y,-x] ,
        }    
        locs = self.beacons
        rotated_locs = [ tuple(rotations[rot_type](*loc)) for loc in locs  ]
        return rotated_locs

In [9]:
import re
from copy import deepcopy

def read_in_scanners(filepath):
    scanners = []
    with open(filepath, 'r') as f:
        for line in f.readlines():
            line = line.strip()
            if len(line) < 2:
                scanners.append(Scanner(idx=idx, locations=elements))
                continue
            if line[:3] == '---':
                m =re.search('--- scanner (\d+) ---',line)
                idx = int(m.group(1))
                elements = []
                continue
            elements.append(line.strip())
    
    scanners.append(Scanner(idx=idx, locations=elements))
    return scanners

def shift_beacons(beacons, shift):
    return [ tuple([i[0] - i[1] for i in zip(loc,shift)]) for loc in beacons]
    
def align_pair_scanner(ref, other, critical_number = 12):
    ref_beacons = ref.get_beacons()
    break_flag = False
    max_align = 0
    
    for ref_pt in ref_beacons[:-11]:
        for rotation in range(24):
            rotated_beacons = other.perform_rotation(rotation)
            for other_pt in rotated_beacons:
                
                shift = [ i[0] - i[1] for i in zip(other_pt,ref_pt)]
                shifted_locs = shift_beacons(rotated_beacons, shift)
                
                aligned_count = len(set(ref_beacons).intersection(shifted_locs))
                max_align = max(max_align, aligned_count)
                
                if aligned_count >= critical_number:
                    break_flag = True
                    break
            if break_flag:
                break
        if break_flag:
            break
            
    if break_flag:
        return rotation, shift, aligned_count 
    else:
        return max_align

def align_all_scanners(input_scanners):
    scanners = deepcopy(input_scanners)
    aligned_set = [0]
    jumbled = [ i for i in range(1,len(scanners))]
   
    ref_idx =0
    
    while len(jumbled) > 0:
        scan_num = aligned_set[ref_idx]
        ref_scanner = scanners[scan_num]
        for idx in jumbled:
            other_scanner = scanners[idx]
            out_put = align_pair_scanner(ref_scanner, other_scanner )
            if type(out_put) == int:
                continue
            else:
                other_scanner.rotate_beacons(out_put[0])
                other_scanner.shift_beacons(out_put[1])
                other_scanner.set_scanner_location(out_put[1])
                aligned_set.append(idx)
                jumbled = [i for i in range(len(scanners)) if i not in aligned_set]
        ref_idx+=1
    return scanners

def count_beacons_and_dist(input_scanners):
    aligned_scanners = align_all_scanners(input_scanners)
    beacons = set([])
    for i in aligned_scanners:
        beacons = beacons.union(i.get_beacons())
    max_dist =0
    for i in aligned_scanners[:-1]:
        for j in aligned_scanners[1:]:
            man_dist = sum([abs(dim[0]-dim[1]) for dim in  zip(i.scanner_location,j.scanner_location)  ] )
            max_dist = max(max_dist, man_dist)
            
    return len(beacons), max_dist
    

In [10]:
test_scanners = read_in_scanners('data/test_19.txt')

In [11]:
aligned_test = align_all_scanners(test_scanners)

In [12]:
%%time
count_beacons_and_dist(test_scanners)

CPU times: user 627 ms, sys: 2.65 ms, total: 630 ms
Wall time: 628 ms


(79, 3621)

In [13]:
prod_scanners = read_in_scanners('data/input_19.txt')

In [14]:
%%time
count_beacons_and_dist(prod_scanners)

CPU times: user 48.3 s, sys: 108 ms, total: 48.4 s
Wall time: 48.4 s


(425, 13354)